In [1]:
#Importing library
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from glob import glob
from tensorflow.keras.models import load_model

In [2]:
np.random.seed(101)

# useful for getting number of output classes
from google.colab import drive
drive.mount("/content/drive/")
folders = glob('/content/drive/My Drive/db/train/*')
train_path = '/content/drive/My Drive/db/train'
valid_path = '/content/drive/My Drive/db/test'

Mounted at /content/drive/


In [3]:
myModel = Sequential()
myModel.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
myModel.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
myModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
myModel.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='same'))
myModel.add(BatchNormalization())
myModel.add(Activation('relu'))
myModel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
myModel.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
myModel.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
myModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
myModel.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
myModel.add(BatchNormalization())
myModel.add(Activation('relu'))
myModel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
#Passing it to a Fully Connected layer
myModel.add(Flatten())
# 1st Fully Connected Layer
myModel.add(Dense(4096, input_shape=(224,224,3,)))
myModel.add(BatchNormalization())
myModel.add(Activation('relu'))
# Add Dropout to prevent overfitting
myModel.add(Dropout(0.4))
#2nd Fully Connected Layer
myModel.add(Dense(1000))
myModel.add(BatchNormalization())
myModel.add(Activation('relu'))
#Add Dropout
myModel.add(Dropout(0.2))
#Output Layer
myModel.add(Dense(10))
myModel.add(BatchNormalization())
myModel.add(Dense(len(folders), activation='softmax'))
myModel.summary()
myModel.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 96)        743520    
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 96)        0

In [4]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/db/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/db/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2950 images belonging to 3 classes.
Found 690 images belonging to 3 classes.


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = 'saved_model/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [7]:
# fit the model
# Run the cell. It will take some time to execute
mymodel_ready = myModel.fit(
  training_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[model_checkpoint_callback]
)

Epoch 1/15
93/93 [==============================] - ETA: 0s - loss: 0.5567 - accuracy: 0.8454 WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/assets
93/93 [==============================] - 1219s 13s/step - loss: 0.5567 - accuracy: 0.8454 - val_loss: 10.4898 - val_accuracy: 0.2072
Epoch 2/15
93/93 [==============================] - 127s 1s/step - loss: 0.3772 - accuracy: 0.8888 - val_loss: 0.7128 - val_accuracy: 0.7841
Epoch 3/15
93/93 [==============================] - 120s 1s/step - loss: 0.3182 - accuracy: 0.8959 - val_l

In [ ]:
# save it as a h5 file

# Save the entire model as a SavedModel.
myModel.save('saved_model/myModel.h5')
from google.colab import files
files.download("saved_model/myModel.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = load_model('/content/drive/MyDrive/db/models/myModel.h5')
model.get_config()

{'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'name': 'conv2d_30_input',
    'ragged': False,
    'sparse': False}},
  {'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 224, 224, 3),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'dtype': 'float32',
    'filters': 64,
    'groups': 1,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'conv2d_30',
    'padding': 'same',
    'strides': (1, 1),
    'trainable': True,
    'use_bias': True}},
  {'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constrain

In [ ]:

model.evaluate()